## Logic Rule Simulation

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [1]:
import os
import subprocess
from pymavlink import mavutil
import numpy as np
from collections import OrderedDict

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns
from helpers.gazebo_world import update_world
from helpers.math import manhattan_distance
from helpers.navegation_logic import find_path,find_best_waypoint

## Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/attack_city.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

# Choose Simulator

In [4]:
simulator ='gazebo' # 'QGroundControl' #

# Choose Initial Position

In [5]:
offsets = [(40, -10, 0, 0),(40, 20, 0, 0)]#
target_pos = [(0, 4, 0),(0, 4, 0)]#
init_alt=[5,5]#,5
n_uavs = len(offsets)
waypoints = OrderedDict({(40, -10, 5):'green',
                        (15, -10, 5):'green',
                        (0, -10, 5):'green',
                        (0, 4, 5):'green',
                       (40, 20, 5):'green',
                        (15, 20, 5):'green',
                        (0, 20, 5):'orange'})


                # lat,long,alt,heading
if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,90)
    spawns=find_spawns(home_position, offsets)

In [6]:
waypoints_np = np.array(list(waypoints.keys()))  # Extract keys as a list of tuples
offsets_np=np.array(offsets)[:,:3]
targets_np=np.array(target_pos)
def get_uav_wps(waypoints: np.ndarray, offsets: np.ndarray) -> np.ndarray:
    """Computes UAV waypoints using NumPy broadcasting."""
    uav_wps =   waypoints[None, :, :] -offsets[:, None, :]
    return uav_wps

uav_wps=get_uav_wps(waypoints_np, offsets_np)
uav_target=targets_np - offsets_np

In [7]:
uav_target

array([[-40,  14,   0],
       [-40, -16,   0]])

## Waypoint seletion algorithm

In [8]:
# Compute path

uavs_wps_2d=[]
uavs_target_2d=[]
for i in range(n_uavs):
        wps_2d=uav_wps[i][:,:2]
        uav_wps_2d=[tuple(wp_2d) for wp_2d in wps_2d if not np.array_equal(wp_2d, np.array([0, 0]))]
        uav_target_2d=tuple(uav_target[i][:2])
        print(f'uav-{i} Best path: {find_path(start=(0,0),target= uav_target_2d, waypoints=uav_wps_2d.copy())}') 
        uavs_wps_2d.append(uav_wps_2d)
        uavs_target_2d.append(uav_target_2d)


uav-0 Best path: [(0, 0), (-25, 0), (-40, 0), (-40, 14)]
uav-1 Best path: [(0, 0), (-25, 0), (-40, 0), (-40, -16)]


## Launch Simulator

In [9]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,waypoints,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

## Create Plan

In [10]:
plan=['check_prearm','check_pos_est','mode_stabilize','mode_guided','arm','takeoff','fly','land']
uavs_arg = []
altitude = 5
path_2d= [(0, 0),(-20, -10)]#      
for i in range(n_uavs):
    uav_arg={'waypoints':[(0, 0,init_alt[i]),tuple(uav_target[i])],
             'plan': plan,
             'wp_margin': 0.5}
    uavs_arg.append(uav_arg)

# Launch Vehicle

In [11]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)

In [12]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

## Create UavLogic

In [13]:
uavs=[]
for i,uav_arg in enumerate(uavs_arg):
    connection = mavutil.mavlink_connection(f'udp:127.0.0.1:{14551+10*i}')
    connection.wait_heartbeat()
    uav=VehicleLogic(connection,**uav_arg)
    uavs.append(uav)

vehicle 1 created
vehicle 2 created


In [14]:
uav_arg

{'waypoints': [(0, 0, 5), (-40, -16, 0)],
 'plan': ['check_prearm',
  'check_pos_est',
  'mode_stabilize',
  'mode_guided',
  'arm',
  'takeoff',
  'fly',
  'land'],
 'wp_margin': 0.5}

## Execute Plan

In [15]:
mission_incomplete = n_uavs*[True]
target_reached =n_uavs*[False] 
waypoints=uavs_wps_2d
wp=n_uavs*[0]
while any(mission_incomplete):
    for i,uav in enumerate(uavs):
        mission_incomplete[i] = uav.act_plan()
        if uav.is_reached and (not target_reached[i]):
            wp[i]+=1
            target=uavs_target_2d[i]
            current = uav.get_local_position()[:2]
            # GPS spoofing
            # if manhattan_distance(current,(-25, 0))<1:
            #     current=(-25, -20)
            best_waypoint = find_best_waypoint(current, target, waypoints[i])
            waypoints[i].remove(best_waypoint) 
            path_2d=[current,best_waypoint]
            if best_waypoint!=target:
                path_2d.append(target)
            else:
                target_reached[i]=True
            print(f'--- waypoint {wp[i]} reached --- ')
            print(f'current position: ({path_2d[0][0]:.2f},{path_2d[0][1]:.2f})')
            print(f'next waypoint: {path_2d[1]}')
            print(f'target position: {path_2d[-1]}')
            path = [(x, y, altitude) for x, y in path_2d]
            uav.update_waypoints(path)
        

vehicle 1: action land is done
vehicle 2: action land is done
vehicle 1: action check_prearm is done
vehicle 2: action check_prearm is done
vehicle 1: action check_pos_est is done
vehicle 2: action check_pos_est is done
vehicle 1: action mode_stabilize is done
vehicle 2: action mode_stabilize is done
vehicle 1: action mode_guided is done
vehicle 2: action mode_guided is done
vehicle 1: action arm is done
vehicle 2: action arm is done
--- waypoint 1 reached --- 
current position: (0.01,0.01)
next waypoint: (-25, 0)
target position: (-40, 14)
--- waypoint 1 reached --- 
current position: (0.01,0.01)
next waypoint: (-25, 0)
target position: (-40, -16)
vehicle 1: action takeoff is done
[(0.011977975256741047, 0.014475650154054165, 5), (-25, 0, 5), (-40, 14, 5)]
vehicle 2: action takeoff is done
[(0.011654230765998363, 0.012608680874109268, 5), (-25, 0, 5), (-40, -16, 5)]
[(0.011977975256741047, 0.014475650154054165, 5), (-25, 0, 5), (-40, 14, 5)]
[(0.011654230765998363, 0.01260868087410926

## Kill all related process

In [16]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")